In [1]:
# Import required libraries
import google.cloud.bigquery as bq
from google.cloud import storage
import pandas as pd
import json


# Initialize BigQuery client
client = bq.Client()
storage_client = storage.Client()

# Configuration
PROJECT_ID = client.project
DATASET_ID = 'EA_DEMO_RAW'
BUCKET_NAME = f'ea-demo-1raw'


print(f'The project currently set is :{PROJECT_ID}')

The project currently set is :xm497-2025-09-16-sqvyh-1


In [2]:
# Load  data from CSV
def load_data():
    """Load books data from CSV into T_BOOKS_RAW table"""

    table_id = f"{PROJECT_ID}.{DATASET_ID}.T_BOOKS_RAW"

    # Configure the load job
    job_config = bq.LoadJobConfig(
        source_format=bq.SourceFormat.CSV,
        skip_leading_rows=1,  # Skip header row
        autodetect=False,  # Don't auto-detect schema, use existing table schema
        write_disposition=bq.WriteDisposition.WRITE_TRUNCATE,  # Replace existing data
        max_bad_records=0  # Fail if any bad records
    )

    # GCS URI for books CSV file
    uri = f"gs://{BUCKET_NAME}/books.csv"

    print(f" Loading books data from: {uri}")
    print(f" Target table: {table_id}")

    # Start the load job
    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )

    # Wait for job to complete
    load_job.result()

    # Get the updated table info
    table = client.get_table(table_id)
    print(f"Loaded {table.num_rows:,} rows into T_BOOKS_RAW")
    return load_job




In [3]:
books_job=load_data()

 Loading books data from: gs://ea-demo-1raw/books.csv
 Target table: xm497-2025-09-16-sqvyh-1.EA_DEMO_RAW.T_BOOKS_RAW
Loaded 100 rows into T_BOOKS_RAW


In [4]:
%%bigquery Books_count
select count(*) as cnt from `xm497-2025-09-16-sqvyh-1`.EA_DEMO_RAW.T_BOOKS_RAW

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:

if  Books_count['cnt'][0] == 100:
    print("The Books CSV data load is done as expected")
else :
    print("The Books CSV data load is not done as expected")

The Books CSV data load is done as expected
